<a href="https://colab.research.google.com/github/OBB-2199/EscapeEarth/blob/main/Interns/Olivia/run_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## mount google drive to access files

from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
!pip install lightkurve==1.9.0
import lightkurve as lk
import pandas as pd
import sys
sys.path.append('/content/gdrive/My Drive/EscapeEarthData/')
import OpenAndPlot as op

     |████████████████████████████████| 501kB 5.7MB/s 
     |████████████████████████████████| 6.5MB 14.3MB/s 
     |████████████████████████████████| 256kB 42.1MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 3.2MB 52.3MB/s 
  Created wheel for fbpca: filename=fbpca-1.0-cp37-none-any.whl size=11376 sha256=ab62631cc4c814761afe914f1297fb7311bd57a20a96f47512f39bc4cae8d023
  Stored in directory: /root/.cache/pip/wheels/53/a2/dd/9b66cf53dbc58cec1e613d216689e5fa946d3e7805c30f60dc
  Created wheel for astroquery: filename=astroquery-0.4.1-cp37-none-any.whl size=3831874 sha256=3f314df30f85ae4e99ac0ba28807f4ad34eab393115e756219378c71080caea3
  Stored in directory: /root/.cache/pip/wheels/88/f8/b7/a254cd96e808f708bc0b7d755a8e095c56fbbe94099d7b464f
  Created wheel for oktopus: filename=oktopus-0.1.2-cp37-none-any.whl size=12780 sha256=57dc7e507f9761de8802bfe533c48561c41ae9f2bebe2831e55e08dc89081509
  Stored in directory: /root/.cache/pip/wheels/9

In [ ]:
#https://docs.google.com/presentation/d/1nJR1YiOolG41xITh9mAs61WhhRY_JuTn8uIx49LPm5k/edit#slide=id.p

def periods(N=1000):
    period=np.logspace(-0.523, 1.43, N, endpoint=True)
    return period

def duration_grid(N=10):
    duration=np.linspace(.01, 0.298, N)
    return duration

def flatten (lc):
    '''
    inputs: cleaned lightkurve data
    output: flattened time and flux data
    function: takes in lightkurve data and flattens the time and flux
    '''
    lc_flat = lc.flatten()
    flat_time = lc_flat.time - lc_flat.time[0]
    return lc_flat, flat_time

def BLS(periodgrid,lightcurve,flat_time,durationgrid):
    from astropy.timeseries import BoxLeastSquares

    '''
    Purppose
    ------------------
    A Box Least Squares function to print out the compute stats of the periodogram.

    Parameters
    -------------------
    period grid            - describes how often the transit is happening (arrays different value)
    duration grid          - describes the width of the transit (array of different values)
    lightcurve             - lightkurve class object

    Returns
    list of stats in the following order: period, duration, transit-time, power, depth
    ------------------
    Calculate several statistics of a candidate transit.
    '''
    #assigning parameters to variables
    period = periodgrid
    duration = durationgrid
    lc = lightcurve

    t = flat_time #time
    y = lc.flux #flux

    #dy is the uncertianty
    model = BoxLeastSquares(t, y, dy= lc.flux_err)
    periodogram = model.power(period,duration)
    max_power = np.argmax(periodogram.power)

    #calculates the max stats w/in the transit
    stats = [periodogram.period[max_power],
            periodogram.duration[max_power],
            periodogram.transit_time[max_power],
            max_power,
            periodogram.depth[max_power]]

    #stats is the one peak, periodogram is the array
    return stats
##############################
##############################
#change sector here
sector = 14
##############################
##############################


#opening target lists
path14 = '/content/gdrive/My Drive/EscapeEarthData/all_targets_S014_v1.csv'
path15 = '/content/gdrive/My Drive/EscapeEarthData/all_targets_S015_v1.csv'
Olivia_path14 = '~/Users/brownscholar/Desktop/Brown Scholars/Internship/all_targets_S014_v1.csv'
Olivia_path15 = '~/Users/brownscholar/Desktop/Brown Scholars/Internship/all_targets_S015_v1.csv'

target_list14 = pd.read_csv(path14,skiprows=5) #from sector 14
target_list15 = pd.read_csv(path15,skiprows=5) #from sector 15
testing_targets = [ 7582633, 7620704, 7618785 ] #7582594, 7584049 – don't exist


In [ ]:
save_power = '/content/gdrive/My Drive/EscapeEarthData/TeamB/bls_powers-RunTeam.npy'
save_period = '/content/gdrive/My Drive/EscapeEarthData/TeamB/bls_period-RunTeam.npy'
save_duration = '/content/gdrive/My Drive/EscapeEarthData/TeamB/bls_duration-RunTeam.npy'
save_depth = '/content/gdrive/My Drive/EscapeEarthData/TeamB/bls_depth-RunTeam.npy'
save_transit_time = '/content/gdrive/My Drive/EscapeEarthData/TeamB/bls_transit_time-RunTeam.npy'
save_path = [save_depth,save_duration,save_transit_time,save_period,save_power]

final_save = '/content/gdrive/My Drive/EscapeEarthData/TeamB/bls_stats_df.csv'

In [ ]:
#period grid
pg = periods()
#duration grid
dg = duration_grid()
#create a empty list for each stat (period,depth,duration,power,transit_time)
period = []; depth = []; duration = []; power =[]; transit_time = []

#change targets here
targets = testing_targets

#have to run for each sector
for star_id in targets:
    data = [star_id,sector]
    #intialize class
    lc = op.OpenAndPlot(data)
    #open data
    open_data = lc.open_lc('clean')
    #flatten the data
    flat_lc,flat_time = flatten(open_data)
    #run the bls on flattened data
    bls_output = BLS(pg,flat_lc,flat_time,dg)
    #append each of the stats to a separate list
    period.append(bls_output[0]); duration.append(bls_output[1]) 
    transit_time.append(bls_output[2]); power.append(bls_output[3])
    depth.append(bls_output[4])
    #save that list
    #np.save()
    my_arr = [depth,duration,transit_time, period, power]
    for i in range(len(save_path)):
      np.save(save_path[i],my_arr[i])
    
print(len(period),len(duration),len(depth),len(power),len(transit_time))
print(period,duration,depth,depth,power,transit_time)

#create a dictionary of all the stats
stats_dict = {'Period':period,'Duration':duration,'Transit Time':transit_time, "Power": power,'Depth':depth}
#use that dictionary to create a pandas dataframe
stats_df = pd.DataFrame(stats_dict)
#write that dataframe to a file
stats_df.to_csv(final_save)

3 3 3 3 3
[0.2999162518987651, 26.915348039269155, 0.3067430805462068] [0.298, 0.202, 0.298] [0.00039819819922779106, 0.007276056431156272, 2.4909213180129014e-05] [0.00039819819922779106, 0.007276056431156272, 2.4909213180129014e-05] [0, 999, 5] [0.157, 26.769, 0.163]
